In [12]:
import numpy as np
from helpers.activfunc import ReLu, sigmoid
from helpers.lossfunc import sum_squares, cross_entropy
from helpers.regularizer import l2_regularizer, l1_regularizer # make an instance when calling train_model
from helpers.grad_descent import SGD, MiniBatchGD
from helpers.accelerator import adam, rmsprop, momentum

In [13]:
activfunc_dict = {
    'relu': ReLu, 'sigmoid': sigmoid
}

lossfunc_dict = {
    'sumsquares': sum_squares, 'crossentropy': cross_entropy
}

grad_descent_dict = {
    'sgd': SGD, 'minibatchgd': MiniBatchGD
}

accelerator_dict = {
    'adam': adam, 'rmsprop': rmsprop, 'momentum': momentum
}

regularizer_dict = {
    'l2': l2_regularizer, 'l1': l1_regularizer
}

In [15]:
class neuralnetwork:
    
    def __init__(self):
        self.W = []
        self.Z = []
        self.Y = []
        self.Wgrad = None
        self.nlayers = 0
        self.dropouts = []
        self.activations = []
        self.prev_layer_neurons = None
        
    def add_layer(self, num_neurons, activation, dropout = None):
        if self.nlayers == 0:
            self.W.append(None)
        else:
            weights = np.random.rand(num_neurons, self.prev_layer_neurons + 1)
            self.W.append(weights)
        
        self.dropouts.append(dropout)
        self.activations.append(activation.lower())
        self.prev_layer_neurons = num_neurons
        self.nlayers += 1
    
    def forward(self, inputs, predict = False):
        
        current_x = inputs
        np.insert(current_x, 0, 1, axis = 0)
        self.Y.append(current_x)
        for k in range(nlayers):
            z = (np.matmul(self.W[k], current_x.T)).T
            Z.append(z)
            
            sigma = activfunc_dict[self.activations[k]]
            y = sigma.forward(z)
            if predict == False and self.dropouts[k] != None:
                 y /= self.dropouts[k]
            self.Y.append(y)
            current_x = y
            np.insert(current_x, 0, 1, axis = 0)
                        
        return y
        
    def backward(self, op_gradient, regularizer):
        current_grad = op_gradient
        for k in range(nlayers, -1, -1): 
            sigma = activfunc_dict[self.activations[k - 1]]
            gradDz = np.matmul(current_grad, sigma.backward(self.Z[k - 1]))
            gradzw = np.matmul(np.ones(np.shape(W[k - 1])[0]), self.Y[k - 1])
            self.Wgrad[k - 1] = np.matmul(gradDz, gradzw)
            if regularizer != None:
                self.Wgrad[k - 1] += regularizer.gradient(self.W[k - 1])
            current_grad = np.matmul(gradDz, self.W[k - 1]) 
    
    def clear_outputs(self):
        self.Z = []
        self.Y = []
        
    def train_network(self, X_train, Y_train, loss_function, grad_descent_type = 'minibatchgd',  
                      learning_rate = 0.001, regularizer = None, accelerator = None):
        
        input_size = np.shape(X_train)[1]
        layer1_size = np.shape(self.W[1])[1]
        self.W[0] = np.random.rand(layer1_size, input_size + 1)
        self.Wgrad = np.zeros(np.shape(self.W))
        loss_function = lossfunc_dict[loss_function.lower()]
        regularizer = regularizer_dict[regularizer.lower()]
        accelerator = accelerator_dict[accelerator.lower()]
        grad_descent = grad_descent_dict[grad_descent_type.lower()]
        grad_descent(self, X_train, Y_train, loss_function, learning_rate, regularizer, accelerator)

    def predict(self, X_test):
        
        Y_test_pred = []
        for x in X_test:
            y = self.forward(x, predict = True)
            Y_test_pred.append(y)
            
        return Y_test_pred
       